In [209]:
!pip install flair

In [210]:
import pandas as pd
import flair
import re

# Imports to get working dates of US
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay

Importing tweets

In [211]:
df=pd.read_csv('10Y_tweets.csv', error_bad_lines=False, sep='\t', lineterminator='\r')
display(df)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,9,21) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,tweet,language,mentions,urls,photos,replies_count,retweets_count,likes_count,hashtags,cashtags,link,retweet,quote_url,video,thumbnail,near,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,1410651811145474052,1.410652e+18,2021-07-01 13:29:35 EDT,2021-07-01,13:29:35,-400.0,8.609864e+17,realheel80,Jason Abbott,NaN,Watching for a ttm squeeze setup in US10Y yiel...,en,[],[],['https://pbs.twimg.com/media/E5Oj9QmWUAgvnWz....,0.0,0.0,0.0,[],[],https://twitter.com/RealHeel80/status/14106518...,False,NaN,1.0,https://pbs.twimg.com/media/E5Oj9QmWUAgvnWz.png,NaN,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
1,1410641500661174276,1.410642e+18,2021-07-01 12:48:36 EDT,2021-07-01,12:48:36,-400.0,3.751287e+08,kikitaly,GolDenRelax - beware of cycles !,NaN,10Y YELD USA a me sembraa una BULLFLAG!! - #US...,es,[],['http://it.tradingview.com/chart/US10Y/BeM3CM...,[],0.0,0.0,0.0,['us10y'],[],https://twitter.com/kikitaly/status/1410641500...,False,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
2,1410631729774088193,1.410632e+18,2021-07-01 12:09:47 EDT,2021-07-01,12:09:47,-400.0,1.255178e+18,bluestonefxfx,ブルーストーン,NaN,【足元の相関】 ナス・・・反転上昇 金・・・・反転上昇 ドル・・・反転下落 利回り・・反転...,ja,[],[],['https://pbs.twimg.com/media/E5OSFLiVcAAm07S....,0.0,0.0,1.0,"['nas100', 'gold', 'dxy', 'us10y']",[],https://twitter.com/BlueStoneFXfx/status/14106...,False,NaN,1.0,https://pbs.twimg.com/media/E5OSFLiVcAAm07S.jpg,NaN,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
3,1410630217354383367,1.410630e+18,2021-07-01 12:03:46 EDT,2021-07-01,12:03:46,-400.0,3.128603e+08,rosentalinv,Rosental Inversiones,NaN,🌎📊 Los rendimientos de los #bonos del Tesoro d...,es,[],[],[],1.0,0.0,0.0,"['bonos', 'eeuu', 'us10y']",[],https://twitter.com/Rosentalinv/status/1410630...,False,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
4,1410620520777719813,1.410621e+18,2021-07-01 11:25:14 EDT,2021-07-01,11:25:14,-400.0,1.255178e+18,bluestonefxfx,ブルーストーン,NaN,【足元の相関】 ナス・・・急落 金・・・・急落 ドル・・・急騰 利回り・・急騰 引き続き...,ja,[],[],['https://pbs.twimg.com/media/E5OHweHUYAQIULn....,0.0,0.0,1.0,"['nas100', 'gold', 'dxy', 'us10y']",[],https://twitter.com/BlueStoneFXfx/status/14106...,False,NaN,1.0,https://pbs.twimg.com/media/E5OHweHUYAQIULn.jpg,NaN,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49143,\n7473696793,7.473697e+09,2010-01-07 04:28:59 EST,2010-01-07,04:28:59,-400.0,2.027491e+07,tradingfloorcom,TradingFloor,NaN,#top10picks Long US10Y and short JGB http://o...,in,[],[],[],0.0,0.0,0.0,['top10picks'],[],https://twitter.com/TradingFloorcom/status/747...,False,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
49144,\n4529243561,4.529244e+09,2009-10-01 13:01:16 EDT,2009-10-01,13:01:16,-400.0,1.487465e+07,jschulmansr,jschulmansr,NaN,"TREASURIES-Bonds rally on stock sell-off, reco...",en,[],[],[],0.0,0.0,0.0,[],[],https://twitter.com/jschulmansr/status/4529243561,False,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
49145,\n4525753389,4.525753e+09,2009-10-01 10:31:55 EDT,2009-10-01,10:31:55,-400.0,1.487465e+07,jschulmansr,jschulmansr,NaN,"TREASURIES-Bonds rise on weaker stocks, jobles...",en,[],[],[],0.0,0.0,0.0,[],[],https://twitter.com/jschulmansr/status/4525753389,False,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
49146,\n1959605843,1.959606e+09,2009-05-29 09:34:33 EDT,2009-05-29,09:34:33,-400.0,1.542268e+07,siriusforex,Sirius Forex Trading Group,NaN,#siriusforex Free Forex Trading Tips Online: D...,en,[],[],[],0.0,0.0,0.0,['siriusforex'],[],https://twitter.com/siriusforex/status/1959605843,False,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN


Keeping only tweets in English

In [212]:
df = df[df['language'] == 'en']
df.reset_index(inplace=True)
del df['index']
df1 = df[['id','date', 'created_at', 'tweet']]
display(df1)

,id,date,created_at,tweet
0,1410651811145474052,2021-07-01,2021-07-01 13:29:35 EDT,Watching for a ttm squeeze setup in US10Y yiel...
1,1410591841112793096,2021-07-01,2021-07-01 09:31:17 EDT,ALERT! Crude Oil Now OVER $75 A Barrel: We Are...
2,1410576445282426886,2021-07-01,2021-07-01 08:30:06 EDT,"#USD Continuing Jobless Claims ➜ Actual 3,469K..."
3,1410576445114617860,2021-07-01,2021-07-01 08:30:06 EDT,#USD Initial Jobless Claims ➜ Actual 364K vs 3...
4,1410575172650864646,2021-07-01,2021-07-01 08:25:03 EDT,#USD Continuing Jobless Claims ➜ in 5 minutes....
...,...,...,...,...
27191,\n11711992147,2010-04-06,2010-04-06 14:32:21 EDT,US10Y just under 4%. Bonds to stocks. Ka ching...
27192,\n9495497367,2010-02-22,2010-02-22 17:03:41 EST,@alaidi thanks ashraf. US10Y at 3.20?
27193,\n4529243561,2009-10-01,2009-10-01 13:01:16 EDT,"TREASURIES-Bonds rally on stock sell-off, reco..."
27194,\n4525753389,2009-10-01,2009-10-01 10:31:55 EDT,"TREASURIES-Bonds rise on weaker stocks, jobles..."


Fetching pre-trained NLP model from Flair

In [213]:
sentiment_model = flair.models.TextClassifier.load('en-sentiment')

2021-07-19 11:59:14,221 loading file /root/.flair/models/sentiment-en-mix-distillbert_4.pt


Removing irrelavent symbols, spaces, notation etc. from the tweets

In [214]:
def clean(tweet):
  whitespace = re.compile(r"\s+")
  web_address = re.compile(r"(?i)http(s):\/\/[a-z0-9.~_\-\/]+")
  user = re.compile(r"(?i)@[a-z0-9_]+")

  # we then use the sub method to replace anything matching
  tweet = whitespace.sub(' ', tweet)
  tweet = web_address.sub('', tweet)
  tweet = user.sub('', tweet)

  tweet = re.sub("\s+", " ", tweet)
  return tweet

df1 = df1[0:len(df1)-1]
df1['tweet'] = df1['tweet'].apply(clean)

Analysing each tweets and adding its probability of being positive

In [215]:
sentiments = []
probabilities = []
i = 0
df1.reset_index(inplace=True)
del df1['index']
for tweet in df1['tweet']:
  if i%1000 == 0:
    print('Processing', i)
  i += 1
  sentence = flair.data.Sentence(tweet)
  sentiment_model.predict(sentence)

  value = sentence.labels[0].value
  score = sentence.labels[0].score
  if value == 'NEGATIVE':
    score = 1 - score
  
  probabilities.append(score)
  sentiments.append(value)

df1['sentiment'] = sentiments
df1['probability'] = probabilities

display(df1)

Processing 0
Processing 1000
Processing 2000
Processing 3000
Processing 4000
Processing 5000
Processing 6000
Processing 7000
Processing 8000
Processing 9000
Processing 10000
Processing 11000
Processing 12000
Processing 13000
Processing 14000
Processing 15000
Processing 16000
Processing 17000
Processing 18000
Processing 19000
Processing 20000
Processing 21000
Processing 22000
Processing 23000
Processing 24000
Processing 25000
Processing 26000
Processing 27000


,id,date,created_at,tweet,sentiment,probability
0,1410651811145474052,2021-07-01,2021-07-01 13:29:35 EDT,Watching for a ttm squeeze setup in US10Y yiel...,NEGATIVE,0.085647
1,1410591841112793096,2021-07-01,2021-07-01 09:31:17 EDT,ALERT! Crude Oil Now OVER $75 A Barrel: We Are...,NEGATIVE,0.259400
2,1410576445282426886,2021-07-01,2021-07-01 08:30:06 EDT,"#USD Continuing Jobless Claims ➜ Actual 3,469K...",NEGATIVE,0.000733
3,1410576445114617860,2021-07-01,2021-07-01 08:30:06 EDT,#USD Initial Jobless Claims ➜ Actual 364K vs 3...,NEGATIVE,0.003755
4,1410575172650864646,2021-07-01,2021-07-01 08:25:03 EDT,#USD Continuing Jobless Claims ➜ in 5 minutes....,NEGATIVE,0.014949
...,...,...,...,...,...,...
27190,\n16784211154,2010-06-22,2010-06-22 12:56:50 EDT,TREASURIES-Dip in Asia on profit-taking: Ten-y...,NEGATIVE,0.003689
27191,\n11711992147,2010-04-06,2010-04-06 14:32:21 EDT,US10Y just under 4%. Bonds to stocks. Ka ching...,NEGATIVE,0.002087
27192,\n9495497367,2010-02-22,2010-02-22 17:03:41 EST,thanks ashraf. US10Y at 3.20?,POSITIVE,0.674852
27193,\n4529243561,2009-10-01,2009-10-01 13:01:16 EDT,"TREASURIES-Bonds rally on stock sell-off, reco...",NEGATIVE,0.034508


Sorting tweets by date for further analysis

In [216]:
df1['date'] = pd.to_datetime(df1['date'], format='%Y-%m-%d')
df1.sort_values(inplace=True, ascending=False, by='date')
df1.reset_index(inplace=True)
del df1['index']
display(df1)

,id,date,created_at,tweet,sentiment,probability
0,1410651811145474052,2021-07-01,2021-07-01 13:29:35 EDT,Watching for a ttm squeeze setup in US10Y yiel...,NEGATIVE,0.085647
1,1410651811145474052,2021-07-01,2021-07-01 13:29:35 EDT,Watching for a ttm squeeze setup in US10Y yiel...,NEGATIVE,0.085647
2,1410576445282426886,2021-07-01,2021-07-01 08:30:06 EDT,"#USD Continuing Jobless Claims ➜ Actual 3,469K...",NEGATIVE,0.000733
3,1410576445114617860,2021-07-01,2021-07-01 08:30:06 EDT,#USD Initial Jobless Claims ➜ Actual 364K vs 3...,NEGATIVE,0.003755
4,1410575172650864646,2021-07-01,2021-07-01 08:25:03 EDT,#USD Continuing Jobless Claims ➜ in 5 minutes....,NEGATIVE,0.014949
...,...,...,...,...,...,...
27190,\n16784211154,2010-06-22,2010-06-22 12:56:50 EDT,TREASURIES-Dip in Asia on profit-taking: Ten-y...,NEGATIVE,0.003689
27191,\n11711992147,2010-04-06,2010-04-06 14:32:21 EDT,US10Y just under 4%. Bonds to stocks. Ka ching...,NEGATIVE,0.002087
27192,\n9495497367,2010-02-22,2010-02-22 17:03:41 EST,thanks ashraf. US10Y at 3.20?,POSITIVE,0.674852
27193,\n4529243561,2009-10-01,2009-10-01 13:01:16 EDT,"TREASURIES-Bonds rally on stock sell-off, reco...",NEGATIVE,0.034508


Finding the daily sentiment score

In [217]:
df_test = df1[:]
date = df_test['date'][0]
daily_sentiment_score = []
counter = 0
score = 0

for i, data in df_test.iterrows():
  if date == data['date']:
    score += data['probability']
    counter += 1
  else:
    daily_sentiment_score.append([date, score/counter])
    date = data['date']
    score, counter = data['probability'], 1

# Adding last value
daily_sentiment_score.append([date, score/counter])

df_new = pd.DataFrame(daily_sentiment_score, columns=['date', 'probability'])
df_new['date'] = pd.to_datetime(df_new['date'], format='%Y-%m-%d')
df_new = df_new[::-1].reset_index()
del df_new['index']
display(df_new)

,date,probability
0,2009-10-01,0.039402
1,2010-02-22,0.674852
2,2010-04-06,0.002087
3,2010-06-22,0.003689
4,2010-06-29,0.054459
...,...,...
1878,2021-06-27,0.520382
1879,2021-06-28,0.430656
1880,2021-06-29,0.168443
1881,2021-06-30,0.301374


Adding a neutral score for missing dates and keeping only the score of working days

In [218]:
us_business_days = CustomBusinessDay(calendar=USFederalHolidayCalendar())
date_range = pd.date_range('10/1/2009', '7/1/2021', freq=us_business_days)
data_df = pd.DataFrame(date_range, columns=['date'])

new_df = pd.merge(left=data_df, right=df_new, how="left", left_on=['date'], right_on=['date'])
new_df.fillna(0.5, inplace=True)
display(new_df)

,date,probability
0,2009-10-01,0.039402
1,2009-10-02,0.500000
2,2009-10-05,0.500000
3,2009-10-06,0.500000
4,2009-10-07,0.500000
...,...,...
2943,2021-06-25,0.248124
2944,2021-06-28,0.430656
2945,2021-06-29,0.168443
2946,2021-06-30,0.301374


Saving daily score

In [219]:
new_df.to_csv('daily_sentiment_score.csv')

Getting the montly sentiment score

In [222]:
new_df.set_index('date', inplace=True)
new_df = new_df.resample('M').mean()
display(new_df)

,probability
date,
2009-10-31,0.478067
2009-11-30,0.500000
2009-12-31,0.500000
2010-01-31,0.500000
2010-02-28,0.509203
...,...
2021-03-31,0.308857
2021-04-30,0.293299
2021-05-31,0.230066


Extracting the data to CSV

In [224]:
new_df.to_csv('monthly_sentiment_score.csv')